<a href="https://colab.research.google.com/github/DavidCarneros/notebooks_uned/blob/main/AA2_T1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En este tema abordaremos el estudio del método estadístico de *bootstrap* y su aplicación para reducir la varianza de algoritmos de aprendizaje automático, en la técnica de agregación de *bootstrap* llamada *bagging*. De ahí, introduciremos los **bosques aleatorios**, un modelo enormemente exitoso y sencillo de utilizar, pero con particularidades matemáticas que lo hacen muy interesante. 

# Descomposición sesgo/varianza

En Aprendizaje Automático se busca crear modelos que ofrezcan dos características esenciales: ajuste a los datos y generalización. La generalización hace referencia a la capacidad predictiva del modelo para adaptarse de manera adecuada a datos nuevos (nunca antes vistos). 

Las medidas de **sesgo** y **varianza** se relacionan con la capacidad de ajuste y generalización de los modelos. Cuando se consigue un
gran ajuste del modelo, la diferencia entre los datos reales y la estimación del modelo es pequeña. En este caso el sesgo también es pequeño. Sin embargo, estos buenos resultados de ajuste, suelen estar acompañados de un aumento en la complejidad del modelo, y, cuando se aumenta la complejidad del modelo, se vuelve sensible a pequeñas variaciones en los datos de entrada, aumentando así la varianza. 

Por tanto, hay una necesidad de encontrar un balance entre sesgo y varianza, o visto de otra forma, entre el error y la complejidad del modelo. 









## Sesgo, varianza y complejidad del modelo

Antes de abordar el estudio de cualquier modelo, es importante entender que su error puede ser descompuesto en varios elementos. Partimos de una variable respuesta $Y$, un vector de entradas $X$, y un modelo predictivo $\hat f(X)$ que ha sido estimado con un conjunto de entrenamiento $\tau$. La función de pérdida para medir el error cometido entre $Y$ y $\hat f(X)$ se denota como $L(Y, \hat f(X))$, y generalmente se utiliza:

$$L(Y, \hat f(X))= \left\{ \begin{array}{lcc}
             (Y - \hat f(X))^2 & & \text{error cuadrático} \\
             \\ |Y - \hat f(X)| && \text{error absoluto} \\
             \end{array}
   \right.$$

Tanto el error cuadrático como el error absoluto se utilizan para para evaluar el rendimiento de un estimador. Es decir qué error cometen al estimar un valor. En ambos casos, se calculan con el valor real ($Y$) y el valor devuelto por el modelo ($\hat f (X))$.

Por otro lado, a la hora de evaluar la capacidad predictiva de un modelo es importante distinguir entre el **error de entrenamiento** y el **error de prueba**:


*   El **error de prueba** (*test error*), también conocido como error de generalización, es el error promedio que resulta de usar el modelo para predecir nuevas observaciones no usadas para ajustar dicho modelo:

$$Err_{\tau} = E[L(Y, \hat f(X)) | \tau]$$

  Una medida relacionada es el **error de predicción esperado** :

  $$Err = E[L(Y, \hat f(X))] = E[Err_{\tau}]$$

*   El **error de entrenamiento**  se obtiene a partir de los datos usados para el entrenamiento del modelo (error al predecir los mismos datos usados para su ajuste):

$$\bar{err} = \frac{1}{N} \sum ^N _{i=1} L(y_i, \hat f(x_i)) $$

La Figura 1 muestra el error de predicción (curvas de colo rojo claro) $Err_{\tau}$
para 100 conjuntos de entrenamiento simulados con un tamaño de 50 observaciones cada uno. La curva roja más gruesa es la media y, por tanto, una estimación de $Err$. 

<center>
<p>
<img src="https://i.gyazo.com/c38bd8bc4f236c812ff6f0aa981376af.png" width="60%"/>
<figcaption>Figura 1. Comportamiento del error de prueba y de entrenamiento al variar la complejidad del modelo. Las curvas de color azul claro muestran el error de entrenamiento, mientras que las las curvas de color rojo claro muestran el error de prueba para 100 conjuntos de entrenamiento de tamaño
50 cada uno, a medida que aumenta la complejidad del modelo. </figcaption>
</p>
</center>

El objetivo es estimar el error de prueba ($Err_{\tau}$). Sin embargo, hay que tener en cuenta que el error de entrenamiento no es un buen estimador para el error de prueba. Tal y como se muestra en la Figura 7.1, el error de entrenamiento disminuye a medida que se aumenta la complejidad del modelo. Sin embargo, un modelo con un error de cercano a cero estará, muy probablemente, sobreajustado a los datos de entrenamiento, por lo que generalizará mal a nuevas observaciones.

En las secciones posteriores estudiaremos algunas técnicas para estimar el error de prueba de un modelo. En este sentido, es importante distinguir entre dos objetivos:



*   **Selección de modelos**. Estimar el rendimiento de diferentes modelos para
elegir el mejor.
*   **Evaluación del modelos**. Una vez elegido un modelo final, estimar su error de predicción (error de generalización) con nuevos datos.

Cuando disponemos de datos suficientes, el mejor enfoque para ambos problemas es
dividir aleatoriamente el conjunto de datos en tres partes: un conjunto de entrenamiento, un conjunto de validación y un conjunto de prueba:

*  **Conjunto de entrenamiento.** Son los datos con los que se entrenan los modelos (error de entrenamiento). 
*  **Conjunto de validación.** Se utiliza para selecciona el mejor de los modelos entrenados (estimando el error de predicción esperado).
*  **Conjunto de prueba.** Se utiliza para evalúar el modelo seleccionado (estimando el error de generalización).


Existe un dilema en las proporciones de esta división, ya que cuanto mas grande sea el conjunto de test, mas preciso será el cómputo del error de test, pero tendremos menos datos de entrenamiento para construir el modelo. No existen reglas de oro para dividir los datos en los subconjuntos necesarios, ya que esa decisión depende, entre otras cosas de la relación, señal-ruido. Una división típica consiste en utilizar el 50% de los datos para el entrenamiento, y un 25% para los conjuntos de validación y prueba. 

Las técnicas que veremos en este tema están orientadas a situaciones en las que no hay datos suficientes para dividirlos en tres partes. De nuevo, es complicado dar una regla general sobre cuántos datos de entrenamiento son suficientes; entre otras cosas, porque esto depende de la relación señal-ruido, y de la complejidad de los modelos que se ajustan a los datos.

## Descomposición sesgo-varianza
La mayoría de los modelos establecen una relación de $Y$ en términos de $X$ a través de una ecuación del tipo $Y = f(X) + \epsilon$, donde $f$ es una función fija pero desconocida, que representa la información sistemática de las variables explicativas, y $\epsilon$ es una variable aleatoria independiente de $X$ con $E[\epsilon] = 0$ y $Var(\epsilon) = \sigma^2_\epsilon$, que representa una perturbación de la información sistemática de $X$. 

Podemos obtener una expresión para el error de predicción esperado cuando se utiliza el modelo estimado en un punto de entrada $X = x_0$ mediante el error cuadrático:

$$\begin{aligned}
Err(x_0) &= E[(Y -\hat f(x_0))^2 | X = x_0] \\
&=\sigma^2_{\epsilon} + [E \hat f(x_0) - f(x_0)]^2 + E[\hat f(x_0) - E \hat f(x_0)]^2 \\
&=\sigma^2_{\epsilon} + \text{Sesgo}^2(\hat f(x_0)) + \text{Var}(\hat f(x_0))\\
&= \text{Error irreducible } + \text{Sesgo}^2 + \text{Varianza}
\end{aligned}$$

Observamos que la medida del error se ha descompuesto de forma que aparecen tres sumandos, según se detalla a continuación:

*   El primer término, llamado ruido o **error irreducible**, es independiente del estimador, solo depende de $\epsilon$. Representa un error inherente a la estimación, ya que aunque se llegue a estimar $f$ de forma perfecta, siempre existiará el error debido a la perturbación $\epsilon$ de los datos. 
*   El segundo término (**sesgo cuadrático**), mide la cantidad en la que la media de la estimación difiere de la media real.
* El último término, la **varianza**, es la desviación cuadrada esperada de $\hat f(x_0)$ alrededor de su media.

Generalmente, cuanto más complejo sea el modelo $f$, menor será el sesgo (al cuadrado) pero mayor será la varianza. Es importante entender la ecuación anterior, porque es general para cualquier modelo y se puede derivar para cada uno de ellos. 

La Figura 2 esquematiza estos conceptos: entenderla bien es garantía de haber entendido lo que se plantea en este epígrafe.

<center>
<p>
<img src="https://i.gyazo.com/5370313d06fc02562ab63ae0a3c8de75.png" width="50%"/>
<figcaption><strong>Figura 2. Esquema del comportamiento del sesgo y la varianza.</strong> El espacio del modelo (model space) es el conjunto de todas las predicciones posibles del modelo, con el "ajuste más cercano" etiquetado con un punto negro. Se muestra el sesgo del modelo, junto con la varianza,
indicada por el círculo amarillo centrado en el punto negro etiquetado como "ajuste más cercano en la población. También se muestra un ajuste reducido o regularizado (shrunken), que tiene un sesgo de estimación adicional, pero un menor error de predicción debido a su menor varianza.</figcaption>
</p>
</center>

# El método de *bootstrap*

En el marco de los bosques aleatorios, es útil tener nociones de en qué
consiste el método estadístico de *bootstrapping*. El método de *bootstrap* es un procedimiento estadístico que se puede utilizar para cuantificar la incertidumbre asociada con un determinado estimador o modelo de aprendizaje estadístico. Por ejemplo, se puede utilizar para estimar el error estándar (SE) de los coeficientes de un modelo de regresión lineal. Sin embargo, *una de las principales ventajas del enfoque bootstrap*, es su  generalidad, ya que la misma metodología básica puede usarse en una gran variedad de situaciones, sin necesidad de acudir a modelos, expresiones o fórmulas específicas para cada problema.

El método *bootstrap* procede mediante **remuestreo**, es decir, obteniendo un gran número de muestras con reposición a través de algún procedimiento aleatorio que utilice los datos originales observados. 

En la práctica, puede ser aplicado para estimar IC y realizar Test de Hipótesis aunque también con fines exploratorios y descriptivos, como diagnosticar modelos o evaluar la replicabilidad de los resultados de un estudio. Mas allá de la aplicación concreta de que se trate, el procedimiento básico resulta siempre el mismo:

1.   Crear un gran número de sub‐muestras con reposición de los mismos datos. Por ejemplo, 2000 muestras.
2.   Calcular para cada muestra resultante el valor del estadístico en cuestión.


Se obtiene así una aproximación a la distribución de muestreo del estadístico, a partir de la cual podemos construir un IC para dicho estadístico o realizar una prueba de significación.  

## Ejemplo

Supongamos que deseamos invertir una cantidad fija de dinero en dos acciones financieras que generan una rentabilidad de $X$ e $Y$, respectivamente, donde $X$ e $Y$ son cantidades aleatorias. Invertiremos una fracción $\alpha$ de nuestro dinero en $X$, y el restante $1- \alpha$ en $Y$. Dado que existe una cierta variabilidad asociada a los rendimientos de estos dos activos, queremos elegir $\alpha$ para minimizar el riesgo total, o la varianza, de nuestra inversión. Dicho de otro modo, queremos minimizar $Var(\alpha X + (1- \alpha)Y)$.

Se puede demostrar que el valor que minimiza este riesgo viene dado por.

$$\alpha = \frac{\sigma ^2 _{Y} - \sigma_{XY}}{\sigma ^2 _{X} + \sigma ^2 _{Y} - 2\sigma _{XY}}$$

Donde $\sigma ^2 _{X} = Var(X)$, $\sigma ^2 _{Y} = Var(Y)$ y $\sigma _{XY} = Cov(X,Y)$. En realidad, las cantidades $\sigma ^2 _{X}$, $\sigma ^2 _{Y}$ y $\sigma _{XY}$ son desconocidas, pero podemos calcular unas estimaciones, $\hat \sigma ^2 _{X}$, $\hat \sigma ^2 _{Y}$ y $\hat \sigma _{XY}$ utilizando un conjunto de datos que contenga mediciones pasadas de $X$ e $Y$. Por tanto, podemos estimar el valor de $\alpha$ que minimiza la varianza de nuestra inversión utilizando:

$$\hat \alpha = \frac{\hat \sigma ^2 _{Y} - \hat \sigma_{XY}}{\hat \sigma ^2 _{X} + \hat \sigma ^2 _{Y} - 2\hat \sigma _{XY}}$$.


La Figura 3 ilustra este enfoque para estimar α en un conjunto de datos simulados. En cada gráfico, hemos simulado 100 pares de rentabilidades para las inversiones $X$ e $Y$. Utilizamos estos resultados para estimar σ $\sigma ^2 _{X}$, $\sigma ^2 _{Y}$ y $\sigma _{XY}$, que después sustituimos en la ecuación anterior para obtener las estimaciones de $\alpha$. Los valores de $\hat \alpha$ resultantes de las estimaciones oscilan entre $0.532$ y $0.657$. 


<center>
<p>
<img src="https://i.gyazo.com/9c36a496bf9b266a138a9fad4f24b8fe.png" width="60%"/>
<figcaption><strong>Figura 3.</strong> Cada panel muestra 100 rentabilidades simuladas para las inversiones X e Y. De izquierda a derecha y de arriba a abajo, las estimaciones resultantes para $\alpha$ son 0,576, 0,532, 0,657 y 0,651</figcaption>
</p>
</center>

Lo más normal es que se quiera cuantificar la precisión de la estimación de $\alpha$. Para ello, podemos repetimr el proceso de simular 100 observaciones de pares X e Y, y calcular $\hat \alpha$ 1000 veces utilizando la ecuación anterior. Obtendremos así 1.000 valores estimados de $\alpha$, que podemos llamar $\hat \alpha_1, \hat \alpha_2, ..., \hat \alpha_{1.000}$

El panel izquierdo de la Figura 4 muestra un histograma de las estimaciones resultantes.  Para estas simulaciones, los parámetros se fijaron en $\sigma^2_X$ = 1, $\sigma^2_Y= 1,25$ y $\sigma_{XY} = 0,5$, por lo que sabemos que el valor real de $\alpha$ es $0,6$. Este valor se indica con una línea vertical sólida en el histograma. La media de las 1.000 estimaciones de $\alpha$ es:

$$\bar \alpha = \frac{1}{1.000} \sum ^{1.000} _{r=1} \hat \alpha_{r} = 0.5996$$

y la desviación típica de la estimación es:

$$\sqrt { \frac{1}{1.000 - 1} \sum ^{1.000} _{r=1} (\hat \alpha_{r} - \bar \alpha)^2} = 0.083$$

Esto nos da una idea de la precisión de $\hat \alpha: SE(\hat{\alpha}) \approx 0,083$. Así pues, a grandes rasgos, para una muestra aleatoria de la población, esperaríamos que $\hat \alpha$ difiriera de $\alpha$ en aproximadamente $0,08$ unidades, de media.

<center>
<p>
<img src="https://i.gyazo.com/ecdc55535ce29b319a7d48f08d089ec4.png" width="90%"/>
<figcaption><strong>Figura 4.</strong> Izquierda: Histograma de las estimaciones de $\alpha$ obtenidas al generar 1.000 conjuntos de datos simulados a partir de la población real. Centro: Un histograma de las
estimaciones de $\alpha$ obtenidas a partir de 1.000 muestras bootstrap de un único conjunto de datos. Derecha: Las estimaciones de $\alpha$ mostradas en los paneles izquierdo y central se representan como boxplots. En cada panel, la línea rosa indica el valor real de $\alpha$.</figcaption>
</p>
</center>

Sin embargo, en la práctica no se puede aplicar el procedimiento de estimación de $SE(\hat \alpha)$ descrito más arriba, porque para los datos reales no podemos generar nuevas muestras a partir de la población original. En cambio, el enfoque bootstrap permite utilizar un computador para emular el proceso de obtención de nuevos conjuntos de muestras, de forma que podamos estimar la variabilidad de $\hat \alpha$ sin tener que generar muestras adicionales. En lugar de obtener repetidamente conjuntos de datos independientes de la población, se obtienen conjuntos de datos distintos mediante el muestreo repetido de observaciones del conjunto de datos original.

Este enfoque se muestra gráficamente en la Figura 5, con un conjunto de datos simple, $Z$, que contiene sólo $n = 3$ observaciones. Se seleccionan aleatoriamente $n$ observaciones del conjunto de datos para producir un  nuevo conjunto de datos *bootstrap*, $Z^{∗1}$. El muestreo se realiza con reemplazo, lo que significa que la misma observación puede aparecer más de una vez en el conjunto de datos bootstrap. Por ejemplo, $Z^{∗1}$ contiene la tercera observación dos veces, la primera observación una vez y ninguna instancia de la segunda observación. Podemos utilizar $Z^{∗1}$ para producir una nueva estimación de $\alpha$. Este procedimiento se repite un número $B$ grande de veces, con el fin de generar $B$ conjuntos de datos *bootstrap* diferentes, $Z^{∗1}, Z^{∗2}, ..., Z^{∗B}$ y sus correspondientes estimaciones de $\alpha$: $\hat \alpha^{*1}, \hat \alpha^{*2}, ..., \hat \alpha^{*B}$. 

<center>
<p>
<img src="https://i.gyazo.com/2987659d9e3f64b4796e3ebc7c1ae76b.png" width="50%"/>
<figcaption><strong>Figura 5.</strong> Ejemplo gráfico del enfoque bootstrap en una pequeña muestra que contiene n = 3 observaciones. Cada nuevo conjunto de datos bootstrap contiene n observaciones, muestreadas con reemplazo del conjunto de datos original, y se utiliza para obtener una estimación de $\alpha$.</figcaption>
</p>
</center>

Podemos calcular el error estándar de estas estimaciones bootstrap utilizando la siguiente fórmula:

$$SE_{B}(\hat \alpha) = \sqrt{\frac{1}{B-1}\sum ^B _{r=1}(\hat \alpha^{*r} - \frac{1}{B} \sum ^B_{r'=1}\hat \alpha^{*r'})^2}$$

El enfoque bootstrap se ejemplifica en el panel central de la Figura 4, que muestra un histograma de 1.000 estimaciones de $\alpha$, cada una calculada utilizando un conjunto de datos bootstrap distinto. Observa que este histograma es muy similar al del panel de la izquierda, que muestra el histograma de las estimaciones de $\alpha$ obtenidas al generar 1.000 conjuntos de datos simulados a partir de la población real. En concreto, con bootstrap se obtiene una estimación para $\alpha$ de $0,087$, muy próxima a la estimación de $0,083$ obtenida utilizando los 1.000 conjuntos de datos simulados.






# Agregación de *bootstrap*: *bagging*

El método de *bootstrap* fue diseñado para determinar la precisión
de la estimación de un parámetro o de una predicción concreta. Sin
embargo, también es posible utilizarlo para mejorar las propias estimaciones
y predicciones. En esto consiste el *bagging*. La idea clave es
que si promediamos un conjunto de observaciones, podemos reducir la
varianza.

Pese a que el término *bagging* proviene de la contracción de *bootstrap
aggregation*, ha
tenido éxito porque, de alguna manera, es posible concebir el *bagging*
como una técnica basada en "embolsar" de distintas maneras un mismo
conjunto de datos. Así, es frecuente encontrar el verbo *to bag* para
referirse a la aplicación del *bagging*.

## Descripción intuitiva

Si $Z_1, Z_2, ..., Z_n$, son observaciones independientes de un mismo
proceso, $Z$, que tiene varianza $\sigma ^2$, entonces la varianza de la media de ellas: $\frac{1}{n} \sum ^n _{i=1} Z_i$ es $\frac{\sigma ^2}{n}$. Es decir, **la media de un conjunto de observaciones reduce la varianza**.

Por tanto, una forma natural de reducir la varianza y mejorar la
predicción es tomar muchos conjuntos de entrenamiento, construir con
ellos modelos de predicción independientes, y calcular la media de las
predicciones. 

<center>
<p>
<img src="https://i.gyazo.com/ab04804474c3b01b8a862a9b110e67e1.png" width="60%"/>
</center>


Como no suele ser habitual tener muchos conjuntos de entrenamiento
independientes, se usa *bootstrap* para conseguirlos a partir de uno solo
original.

## Aplicaciones de *bagging*

[ESL 8.7] Analicemos primero el problema de la regresión. Supongamos que ajustamos un modelo a los datos de entrenamiento $Z = \lbrace (x_1, y_1), (x_2, y_2), ... , (x_N, y_N)\rbrace$, obteniendo una predicción $\hat f(x)$ para una entrada $x$. La agregación de *bootstrap* hace un promedio de dicha predicción
sobre el conjunto de muestras *bootstrap*, reduciendo así su varianza. Para cada una de las muestras *bootstrap* $Z^{∗b}, b = 1, 2, ... ,B$, se ajusta el modelo, generando una predicción $f^{∗b}(x)$, de forma que la estimación *bagging* viene dada por:

$$\hat f_{bag}(x) = \frac{1}{B}\sum _{b=1}^B \hat f^{*b}(x)$$


[ISLR 8.2] Aunque el *bagging* puede mejorar los resultados de muchos algoritmos de regresión, resulta particularmente interesante en el caso de los árboles de decisión. Para ello, simplemente se construyen $B$ árboles de regresión utilizando $B$ conjuntos de entrenamiento con *bootstrap*, y se promedian las predicciones resultantes. Este procedimiento hace que los árboles crezcan en profundidad y no se podan. De esta forma, cada árbol *bootstrap* incluirá normalmente características diferentes a las del original, y podría tener un número diferente de nodos terminales Por tanto, cada árbol particular tendrá una alta varianza y un sesgo pequeño, pero al promediar la predicción generada por los $B$ árboles se reduce la varianza y se deja el sesgo inalterado. 

Se ha demostrado que la técnica de *bagging* proporciona una mejora importante de rendimiento cuando se combinan cientos o incluso miles de árboles en un único procedimiento.

[ESL 8.7] Supongamos ahora que nuestro árbol es un clasificador que genera salidas de $K$ clases. En este caso, se toma la regla de la mayoría (la clasificación
con mayor número de votos). Para ello, es útil considerar una función vectorial $\hat f(x)$, que produce un único valor $1$ y $K-1$ ceros, tal que $\hat G(x) = arg \;\; max_k \hat f (x)$. Así, la estimación del *bagging*, $\hat f_{bag}(x)$ es un vector con $K$ componentes: $[p_1(x), p_2(x), ..., p_K(x)]$, donde $p_K(x)$ es la proporción de árboles que han predicho la clase $k$ para la entrada $x$. El clasificador de *bagging* selecciona la clase con más "votos" en los $B$ árboles: 

$$\hat G_{bag}(x) = arg \;\; max_k \hat f_{bag} (x)$$

En ocasiones, necesitamos las estimaciones de la probabilidad de cada clase para una entrada $x$, en lugar de las propia clasificación. Puede resultar tentador considerar las proporciones de voto $p_k(x)$ como estimaciones de estas probabilidades, pero con un sencillo ejemplo de clasificación binaria podemos comprobar que no es correcto. Supongamos que la probabilidad real de la clase 1 para $x$ es de $0,75$, y que todos los clasificadores *bagging* predicen un 1. Entonces tendriamos que $p_1(x) = 1$, lo cual sería incorrecto.

No obstante, para muchos clasificadores $\hat G(x)$, se dispone de una función subyacente $\hat f(x)$ que estima las probabilidades de cada clase (por ejemplo, para los árboles de clasificación, las proporciones de clase en el nodo terminal). Una estrategia alternativa cuando se utiliza *bagging* consiste en promediar estos vectores de proporciones de clase en el nodo terminal en lugar de los vectores generados de votos. Esto no sólo produce mejores estimaciones de las probabilidades de clase, sino que también tiende a proporcionar clasificadores *bagging* con menor varianza, especialmente para pequeños valores de $B$.


## Ejemplo: Árboles con datos simulados

[ESL 8.7.1] En en esta sección se estudian las propiedades matemáticas del *bagging* para
un sencillo problema de clasificación, que muestra una idea básica que
subyace a este método: promediar reduce la varianza y deja el sesgo
inalterado en el caso de que se use una función de pérdida basada en
el error cuadrático.

Se ha generado una muestra de tamaño $N = 30$, con dos clases y $p = 5$ características, cada una de las cuales tiene una distribución gaussiana estándar con una correlación por pares de $0,95$.  La respuesta $Y$ ha sido generada de acuerdo con $Pr(Y = 1|x_1 ≤ 0,5) = 0,2$, y $Pr(Y = 1|x_1 > 0,5) = 0,8$. El error de Bayes es de $0,2$. También se ha generado un conjunto de prueba de tamaño $2000$ a partir de la misma población. A continuación, se ajustan árboles de clasificación a la muestra de entrenamiento y a $200$ muestras *bootstrap*. 

La Figura 6 muestra el árbol original y once árboles bootstrap. Observa cómo todos los árboles son diferentes, con distintas características de división y puntos de corte.

<center>
<p>
<img src="https://i.gyazo.com/4f261037aeacb699af07b81de08dd443.png" width="60%"/>
<figcaption><strong>Figura 6.</strong> Árboles bagging en el conjunto de datos simulado. El gráfico superior izquierdo representa el árbol original. Se muestran once árboles obtenidos a partir de muestras bootstrap. Para cada árbol, se anota la división superior.</figcaption>
</p>
</center>

El error de test para el árbol original y los árbol *bagging* se muestra en la Figura 7. En este ejemplo, los árboles tienen una alta varianza debido a la correlación de los predictores. Sin embargo, mediante la técnica de *bagging* se consigue reducir esta varianza y, consecuentemente, el error de test. Por tanto, el *bagging* puede reducir considerablemente la varianza de procedimientos inestables como son los árboles, mejorando así el resultado de la predicción.

<center>
<p>
<img src="https://i.gyazo.com/9bd3d8876495405c76fed8e8b0092dec.png" width="50%"/>
<figcaption><strong>Figura 7.</strong> Curvas de error para el ejemplo de bagging de la Figura 6.  Se muestra el error de test del árbol original y de los árboles bagging en función del número de muestras bootstrap.  Los puntos naranjas se corresponden con el voto de "consenso", mientras que los verdes promedian las probabilidades.</figcaption>
</p>
</center>

Supongamos que las observaciones de entrenamiento $(x_i, y_i), i = 1, ...,N$ se obtienen de manera independiente a partir de una distribución $P$, y consideremos que el estimador ideal del agregado es $f_{ag}(x) = E_P \hat f^∗(x)$. En este caso $x$ es fija y el conjunto de datos bootstrap $Z^*$ está formado por las observaciones $x^∗_i , y^∗_i , i = 1, 2, . ,N$ muestreadas de $P$. Hay que tener en cuenta que $f_{ag}(x)$ es una estimación *bagging*, que extrae muestras *bootstrap* de la población real $P$. Aunque no es una estimación que podamos utilizar en la práctica, resulta interesante para el análisis. Podemos escribir:

$$\begin{aligned}
E_p [Y - \hat f^*(x)]^2 &= E_p[Y - f_{ag}(x) + f_{ag}(x) - \hat f^*(x)]^2 \\
&= E_p[Y - f_{ag}(x)]^2  + E_p[\hat f^*(x) - f_{ag}(x)]^2\\
&\geq E_p[Y - f_{ag}(x)]^2
\end{aligned}$$

El error añadido en el lado derecho se debe a la varianza de $\hat f^{∗}(x)$ alrededor de su media $f_{ag}(x)$. Por lo tanto, la agregación de la población real nunca aumenta el error cuadrático medio. Esto sugiere que el *bagging*, es decir, la extracción de muestras de los datos de entrenamiento, suele reducir el error cuadrático medio.

Este argumento no es válido para la clasificación con pérdida 0-1, debido a que el sesgo y la varianza no son complementarios. En ese caso, el uso de *bagging* para un buen clasificador puede mejorarlo, pero si se utiliza con un clasificador malo, puede empeorarlo aún más. Por ejemplo, supongamos que $Y = 1$ para todo $x$, y que el clasificador $\hat G(x)$ predice $Y = 1$ (para todo x) con probabilidad $0,4$ y produce $Y = 0$ (para todo x) con probabilidad $0,6$. En este caso, el error de clasificación de $\hat G(x)$ es de $0,6$, mientras que el del clasificador *bagging* será de $1,0$.


Hay que tener en cuenta que cuando se "embolsa" un modelo, se pierden sus estructuras más simples, y esto es claramente un inconveniente para la interpretación del modelo. No obstante, algunos métodos más estables, como pueden ser los K-vecinos más cercanos, no suelen verse tan afectados por el *bagging*. 

La Figura 8 muestra un ejemplo en el que el *bagging* no ayuda. Se muestran 100 puntos de datos con dos características y dos clases, separadas por un limite lineal $x_1 + x_2 = 1$ (en gris). Seleccionamos como clasificador $\hat G(x)$ una división en un solo eje, y elegimos la división sobre $x_1$ o $x_2$ que provoca la mayor reducción en el error de clasificación.

La curva azul del panel de la izquierda muestra el límite de decisión obtenido al aplicar la regla de decisión en $B = 50$ muestras *bootstrap*. Esta curva no representa bien el límite real.

<center>
<p>
<img src="https://i.gyazo.com/f8b61bc9e4f7d25785fc034988bbccb9.png" width="70%"/>
<figcaption><strong>Figura 8.</strong> Datos con dos características y dos clases, separadas por un límite lineal. Panel izquierdo: Límite de decisión estimado a partir del bagging de la regla de decisión de un único clasificador, dividido y orientado al eje. Panel derecho: Límite de decisión a partir de la intensificación  de la regla de decisión del mismo clasificador. La intensificación se verá más adelante.</figcaption>
</p>
</center>









# Bosques Aleatorios




Hemos visto que el *bagging* o *bootstrap aggregation* es una técnica para reducir la varianza de una función de predicción que ha sido estimada. El *bagging* funciona particularmente bien con métodos de alta varianza y bajo sesgo, como los árboles de decisión. En la regresión, simplemente ajustamos el mismo árbol de regresión muchas veces a distintas muestras *bootstrap* de los datos de entrenamiento y promediamos el resultado. En la clasificación, un grupo de árboles votan por la clase predicha. 

Sin embargo, este procedimiento presenta un problema: la posibilidad de correlación, más o menos alta, de los árboles construídos. Si existe un predictor muy asociado con la respuesta, éste será el causante de la partición del nodo raíz en casi todos los árboles, y consecuentemente, todos los árboles generados serán más o menos parecidos (correlacionados).

El método de los **bosques aleatorios** (*random forests*) es una inteligente modificación del método
de *bagging* que consiste en construir una colección grande de árboles
no-correlados entre sí mediante la introducción de perturbaciones
aleatorias en el proceso inductivo, para luego promediar sus resultados. En otras palabras, los bosques aleatorios son una variante del *bagging* específicamente diseñados para trabajar con árboles de decisión, pues tratan de paliar el problema de correlación entre los árboles.

La principal razón del éxito de los bosques aleatorios es que ofrecen
resultados buenos en multitud de casos, y son muy sencillos de construir
y entrenar.



## Definición de Bosques Aleatorios

Las muestras *bootstrap* que se generan al hacer *bagging* introducen un elemento de aleatoriedad que en la práctica provoca que todos los árboles sean distintos, pero en ocasiones no son lo suficientemente distintos. Es decir, suele ocurrir que los árboles tengan estructuras muy similares, especialmente en la parte alta, aunque después se vayan diferenciando según se desciende por ellos. 

Promediar variables altamente correladas produce una reducción de la varianza mucho menor que si promediamos variables incorreladas. La solución pasa por añadir aleatoriedad al proceso de construcción de los árboles, para que estos dejen de estar correlados. 

Así, la idea de los bosques aleatorios es mejorar la reducción de la varianza del *bagging* disminuyendo la correlación entre los árboles. Esto se consigue mediante la selección aleatoria de las variables de entrada en el proceso de crecimiento del árbol. Concretamente, en un bosque aleatorio, cuando se va a realizar la partición de un nodo, no son candidatas a la partición todas las variables predictoras: se utilizan solo una muestra aleatoria de ellas. 

Por tanto, la modificación introducida por los bosques aleatorios es que antes de hacer cada uno de los cortes, se seleccionan al azar $m \leq p$ de las varables explicativas como candidatos para la división. Así pues, los bosques aleatorios tienen dos fuentes de aleatorización:

* La que supone el *bootstrap*
* El hecho de tomar muestras aleatorias de las variables a la hora de partir un
nodo

El **algoritmo** para construir bosques aleatorios para Regresión o Clasificación es el siguiente:

1. Para $b=1$ hasta $B$ (número de árboles):

  * Generar una muestra *bootstrap* $Z^∗$ de tamaño $N$ de los datos de entrenamiento originales.
  * Desarrollar un árbol de decisión aleatorio $T_b$ para los datos obtenidos con *bootstrap*, repitiendo recursivamente los siguientes pasos de división para cada nodo terminal del árbol, hasta que se alcance un determinado tamaño mínimo de nodo $n_{min}$ (número mínimo de muestras que debe contener el nodo).
    * Seleccionar aleatoriamente $m$ variables de entre los $p$ predictores disponibles.
    * Entre esas $m$ características, elegir aquella que haga el mejor corte (mayor reducción del error). 
    * Dividir el nodo en dos nodos hijos.

2. Devolver el conjunto de árboles $\lbrace T_b \rbrace ^B _1 $

Por tanto, el número características $m$ y el tamaño mínimo de nodo ($n_{min}$) son hiperparámetro de los bosques aleatorios. Breiman, el creador de este algoritmo, propuso unos valores por defecto que funcionan bastante bien:

* Para clasificación, $m \approx \sqrt p$ y $n_{min} = 1$.
* Para regresión, $m \approx p/3$ y $n_{min} = 5$.
* En general, no son necesarias más de 500 muestras *bootstrap*

No obstante, los mejores valores para estos parámetros dependerán del problema, y deben ser considerados como parámetros de ajuste.

Una vez construido el bosque aleatorio, para hacer una predicción sobre un punto de entrada $x$:

* Para regresión, se toma el promedio de las salidas (predicciones) de todos los árboles:

$$\hat f ^B _{rf}(x) = \frac{1}{B} \sum ^B_{b=1}T_b(x)$$

* Cada árbol da una clasificación (vota por una clase), y el resultado es la clase con mayor número de votos en todo el bosque (*forest*).

$$\hat C ^B _{rf} (x) = \text{majority vote}\; \lbrace \hat C_b(x) \rbrace ^B _1$$

La media de $B$ variables aleatorias i.i.d. (independiente e idénticamente distribuidas), cada una con varianza $\sigma ^2$, tiene una varianza $\frac{1}{B}\sigma ^2$. Si las variables son simplemente i.d. (idénticamente distribuidas, pero no necesariamente independientes) con correlación positiva entre pares $\rho$, la varianza de la media es:

$$\rho \sigma ^2 + \frac{1 - \rho}{B}\sigma^2$$

A medida que $B$ aumenta, el segundo término desaparece, pero el primero se mantiene, por lo que la correlación de los pares de árboles embolsados limita las ventajas de promediar. En el caso de los árboles con *bootstrap*, $\rho$ suele ser pequeña (0,05 o menor; mientras que $\sigma ^2$) no es mucho mayor que la varianza del árbol original.

La reducción de $m$ hará que disminuya la correlación entre cualquier par de árboles del conjunto y, por tanto, según la ecuación anterior, se reducirá la varianza de la media.

## Detalles de los Bosques Aleatorios

Una vez comprendida la descripción de los bosques aleatorios, es importante dedicar tiempo a comprender algunos de sus detalles más relevantes.

### Muestras "fuera de la bolsa"

Una característica importante de los bosques aleatorios es el uso de muestras "fuera de la bolsa" (*out-of-bag*, OOB), que permiten aproximar el error de una forma parecida a como lo haríamos con validación cruzada, pero con la ventaja de que en este caso puede hacerse al mismo tiempo que se construye el modelo.

Los bosques aleatorios se entrenan utilizando *bagging*, de forma que cada nuevo árbol se ajusta a una muestra *bootstrap* del conjunto de entrenamiento: $z_i = (x_i, y_i)$. El estimador OOB del error de predicción se basa en aprovechar las observaciones no incluidas en cada muestra *bootstrap*, es decir,  el error OOB se construye considerando sólo los árboles correspondientes a las muestras *bootstrap* en las que $z_i$ no apareció.

En el ejemplo de la Figura 9, el conjunto de datos original está formado por una serie de pacientes, sin embargo, cada modelo se entrena sólo con las observaciones de su muestra (*bag*). Los pacientes de cada conjunto "fuera de la bolsa" pueden utilizarse para probar sus respectivos modelos.  

<center>
<p>
<img src="https://upload.wikimedia.org/wikipedia/commons/3/36/Sampling_with_replacement_and_out-of-bag_dataset_-_medical_context.jpg" width="80%"/>
<figcaption><strong>Figura 9.</strong> Ejemplo de muestreo de 4 observaciones del conjunto original con reemplazo. Se representan las muestras "fuera de la bolsa". En este caso, solo los pacientes de la muestra *bootstrap* se utilizarían para entrenar el modelo de esa "bolsa".</figcaption>
</p>
</center>

La Figura 10 muestra el error de clasificación de OOB para los datos de *spam*, comparado con el error de test. Aunque aquí se promedian 2.500 árboles, del gráfico se puede deducir que bastaría con unos 200.

<center>
<p>
<img src="https://i.gyazo.com/e06ddcb1bf9d46cf2df02cd1c0317716.png" width="60%"/>
<figcaption><strong>Figura 10.</strong> Error OOB calculado en los datos de entrenamiento, comparado con el error de test calculado en el conjunto de test.</figcaption>
</p>
</center>

### Importancia relativa de las distintas variables

Un gran inconveniente de los procedimientos de ensamblado es que son
una "caja negra". La sencillez explicativa de un árbol de decisión simple se pierde en los bosques aleatorios. Además, al ser métodos no paramétricos, no tenemos parámetros que cuantifiquen el impacto de las distintas predictoras.

Sin embargo, es deseable conocer el impacto de cada variable predictora para poder identificar las variables con mayor capacidad predictiva (más asociadas al
resultado) o, por ejemplo, como método de selección de variables.

Es posible obtener una aproximación a la importancia relativa de las distintas variables que intervienen en el modelo a través de dos medidas de importancia:

- **Importancia de Gini**. Mide la mejora en pureza, medida mediante el índice de Gini, producida por la partición. Así, la importancia de una variable en un árbol se mide como la suma de los decrementos atribuidos a esa variable y la importancia final, como la media en todos los árboles. El gráfico izquierdo de la Figura 11 muestra las importancias de las variables calculadas de este modo para el conjunto de datos `spam`.

- **Aleatorización**. Los bosques aleatorios también hacen uso de las muestras OOB para construir una medida de importancia de la variable. El funcionamiento de esta lógica el la siguiente: primero se escoge el error de clasificación *out-of-bag*, después se toma una variable al azar y se permutan sus valores dentro de los datos de entrenamiento, ocasionando que dicha variable escogida descorrelacione lo aprendido por el modelo. Luego se vuelve a calcular el error OOB, para luego compararlo con el error calculado inicialmente. En consecuencia, si el error cambia, se afirma que dicha variable es importante. Este proceso se repite con todas las variables y luego estas
se ordenan de acuerdo a los cambios que produjeron cada una en los errores OOB.

Aunque las puntuaciones de los dos métodos son similares, las puntuaciones de importancia en el gráfico de la derecha son más uniformes sobre las variables.

<center>
<p>
<img src="https://i.gyazo.com/6bf4b74df33f485285f2921b303aab35.png" width="80%"/>
<figcaption><strong>Figura 11.</strong> Gráficos de importancia de las variables para un bosque aleatorio de clasificación con el conjunto de datos spam. El gráfico de la izquierda basa la importancia en el índice de división de Gini, mientras que el gráfico de la derecha utiliza la aleatorización OOB (se observa que tiende a distribuir las importancias de manera más uniforme).</figcaption>
</p>
</center>


### Sobreentrenamiento en bosques aleatorios.

Cuando el número de variables es grande, pero la fracción de variables relevantes es pequeña, es probable que los bosques aleatorios tengan un mal rendimiento con valores de $m$ pequeños. En estos casos, la probabilidad de que en una división se seleccionen las variables relevantes será pequeña. La Figura 12 muestra los resultados de una simulación que apoya esta afirmación.

A medida que esta probabilidad se reduce, la diferencia entre el *boosting* y los bosques aleatorios aumenta. Cuando el número de variables relevantes aumenta, el rendimiento de los bosques aleatorios es considerablemente robusto ante el aumento del número de variables con ruido. Por ejemplo, con 6 variables relevantes y 100 de ruido, la probabilidad de que se seleccione una variable relevante en cualquier división es de 0,46, suponiendo que $m = \sqrt{(6 + 10)} \approx 10$. Según la Figura 12, esto no perjudica el rendimiento de los bosques aleatorios en comparación con el boosting. Esta robustez se debe en gran medida a la poca dependencia del coste de la clasificación errónea con el sesgo y la varianza de las estimaciones de probabilidad en cada árbol.

<center>
<p>
<img src="https://i.gyazo.com/35c99bd697f35df07dbd8a92f53643a6.png" width="60%"/>
<figcaption><strong>Figura 12. </strong>Comparativa de los bosques aleatorios y la intensificación del gradiente (Tema 2) en problemas con un número creciente de variables con ruido. Los bosques aleatorios utilizan el valor por defecto $m = \sqrt p$. En la parte superior de cada par se muestra la probabilidad de que una de las variables relevantes sea elegida en cualquier división. Los resultados se basan en 50 simulaciones para cada par, con un conjunto de entrenamiento de 300 observaciones, y un conjunto de prueba de 500.</figcaption>
</p>
</center>

Otra cuestión es que los bosques aleatorios "no pueden sobreajustar" los datos. Realmente el aumento de $B$ no provoca que la secuencia de bosques aleatorios se sobreajuste. Como el *bagging*, la estimación del bosque aleatorio (15.2) se aproxima a:

$$\hat f_{rf}(x) = E_{\Theta} T(x;\Theta) = \lim_{B\to\infty} \hat f(x)^B_ {rf}$$

con una media sobre $B$ realizaciones de $\Theta$. Sin embargo, este límite puede sobreajustar los datos; la media de los árboles completamente desarrollados puede dar lugar a un modelo demasiado complejo e introducir una varianza innecesaria. Nuestra experiencia nos dice que el uso de árboles completamente desarrollados rara vez cuesta mucho, y da como resultado un parámetro de ajuste menos. 

La Figura 13 muestra el ligero efecto que produce la regulación de la profundidad en un ejemplo de regresión simple. En los clasificadores, la sensibilidad a la varianza es menor, y por tanto, el sobreajuste pocas veces se observa en la clasificación mediante bosques aleatorios.

<center>
<p>
<img src="https://i.gyazo.com/fed0066b376ad5433e90c490d76c12f3.png" width="60%"/>
<figcaption><strong>Figura 13. </strong>Efecto del tamaño del árbol en el error de una regresión por bosque aleatorio. La profundidad del árbol se controla con el tamaño mínimo de los nodos; cuanto más pequeño sea el tamaño mínimo de los nodos, más profundos serán los árboles. </figcaption>
</p>
</center>

# Bibliografía



*   [ISL] An introduction to statistical learning, G. James, D. Witten, T.
Hastie and R. Tibshirani: http://faculty.marshall.usc.edu/gareth-james/ISL/
*   [ESL] The Elements of Statistical Learning (Second), Tibshirani, Robert J. ; Hastie, Trevor ; Friedman, Jerome ;

* https://rubenfcasal.github.io/aprendizaje_estadistico/bagging-rf-r.html

* https://bookdown.org/content/2031/ensambladores-random-forest-parte-i.html


